# **Amazon Lookout for Equipment**
*Part 1 - Data preparation*

### Notebook configuration update
Let's make sure that we have access to the latest version of the AWS Python packages. If you see a `pip` dependency error, check that the `boto3` version is ok: if it's greater than 1.17.48 (the first version that includes the `lookoutequipment` API), you can discard this error and move forward with the next cell:

In [ ]:
import boto3
print(f'boto3 version: {boto3.__version__} (should be >= 1.17.48 to include Lookout for Equipment API)')

# Restart the current notebook to ensure we take into account the previous updates:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

boto3 version: 1.18.28 (should be >= 1.17.48 to include Lookout for Equipment API)


### Imports

In [1]:
import config
import os
import pandas as pd
import boto3
from botocore.client import ClientError

Check the region and the availability of Amazon Lookout for Equipment in this region:

In [2]:
REGION_NAME = boto3.session.Session().region_name

ssm_client = boto3.client('ssm')
response = ssm_client.get_parameters_by_path(
    Path='/aws/service/global-infrastructure/services/lookoutequipment/regions',
)

available_regions = [r['Value'] for r in response['Parameters']]
if REGION_NAME not in available_regions:
    raise Exception(f'Amazon Lookout for Equipment is only available in {available_regions}')

### Parameters
Let's first check if the bucket name is defined, if it exists and if we have access to it from this notebook. If this notebook does not have access to the S3 bucket, you will have to update its permission:

In [3]:
BUCKET           = config.BUCKET
ASSET_ID         = config.ASSET_ID

PREFIX_TRAINING  = f'{ASSET_ID}/training-data/'
PREFIX_LABEL     = f'{ASSET_ID}/label-data/'

In [4]:
if BUCKET == '<<YOUR_BUCKET>>':
    raise Exception('Please update your Amazon S3 bucket name in the config.py file located at the root of this repository and restart the kernel for this notebook.')
    
else:
    # Check access to the configured bucket:
    try:
        s3_resource = boto3.resource('s3')
        s3_resource.meta.client.head_bucket(Bucket=BUCKET)
        print(f'Bucket "{BUCKET}" exists')
        
    # Expose error reason:
    except ClientError as error:
        error_code = int(error.response['Error']['Code'])
        if error_code == 403:
            raise Exception(f'Bucket "{BUCKET}" is private: access is forbidden!')
            
        elif error_code == 404:
            raise Exception(f'Bucket "{BUCKET}" does not exist!')

Bucket "l4esitewisejjhj-asset1-train-inference" exists


In [5]:
RAW_DATA       = os.path.join('..', 'data', 'raw', ASSET_ID)
TMP_DATA       = os.path.join('..', 'data', 'interim', ASSET_ID)
PROCESSED_DATA = os.path.join('..', 'data', 'processed', ASSET_ID)
LABEL_DATA     = os.path.join(PROCESSED_DATA, 'label-data')
TRAIN_DATA     = os.path.join(PROCESSED_DATA, 'training-data')
INFERENCE_DATA = os.path.join(PROCESSED_DATA, 'inference-data')

os.makedirs(TMP_DATA,         exist_ok=True)
os.makedirs(RAW_DATA,         exist_ok=True)
os.makedirs(PROCESSED_DATA,   exist_ok=True)
os.makedirs(LABEL_DATA,       exist_ok=True)
os.makedirs(TRAIN_DATA,       exist_ok=True)
os.makedirs(INFERENCE_DATA,   exist_ok=True)

ORIGINAL_DATA = f's3://lookoutforequipmentbucket-{REGION_NAME}/datasets/getting-started/lookout-equipment-sdk-5min.zip'

## Downloading data
---
Downloading and unzipping the getting started dataset locally on this instance:

In [6]:
data_exists = os.path.exists(os.path.join(TMP_DATA, 'sensors-data', 'impeller', 'component2_file1.csv'))
raw_data_exists = os.path.exists(os.path.join(RAW_DATA, 'lookout-equipment.zip'))

if data_exists:
    print('Dataset already available locally, nothing to do.')
    print(f'Dataset is available in {TMP_DATA}.')
    
else:
    if not raw_data_exists:
        print('Raw data not found, downloading it')
        !aws s3 cp $ORIGINAL_DATA $RAW_DATA/lookout-equipment.zip
        
    print('Unzipping raw data...')
    !unzip $RAW_DATA/lookout-equipment.zip -d $TMP_DATA
    print(f'Done: dataset now available in {TMP_DATA}.')

Raw data not found, downloading it
download: s3://lookoutforequipmentbucket-us-east-1/datasets/getting-started/lookout-equipment-sdk-5min.zip to ../data/raw/27b53ba8-baa0-4298-809a-0afe9384c280/lookout-equipment.zip
Unzipping raw data...
Archive:  ../data/raw/27b53ba8-baa0-4298-809a-0afe9384c280/lookout-equipment.zip
   creating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/inference-data/
  inflating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/inference-data/inference.csv  
   creating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/label-data/
  inflating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/label-data/labels.csv  
   creating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/schema/
  inflating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/schema/schema.json  
   creating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/
   creating: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/impeller/
  in

## Preparing time series data
---
The time series data are available in the `sensors-data` directory. The industrial asset we are looking at is a [centrifugal pump](https://en.wikipedia.org/wiki/Centrifugal_pump). Such a pump is used to move a fluid by transfering the rotational energy provided by a motor to hydrodynamic energy:

<img src="assets/centrifugal_pump_annotated.png" alt="Centrifugal pump" style="width: 658px"/>

<div style="text-align: center"><i>Warman centrifugal pump in a coal preparation plant application</i>, by Bernard S. Janse, licensed under <a href="https://creativecommons.org/licenses/by/2.5/deed.fr">CC BY 2.5</a></div>

On a pump such as the one displayed in the photo above, the fluid enters at its axis (the black pipe arriving at the "eye" of the impeller. Measurements can be taken around the four main components of the centrifugal pump:
* The **impeller** (hidden into the round white casing above): this component consists of a series of curved vanes (blades)
* The drive **shaft** arriving at the impeller axis (the "eye")
* The **motor** connected to the impeller by the drive shaft (on the other end of the black pipe above)
* The **volute** chamber, offseted on the right compared to the impeller axis: this creates a curved funnel win a decreasing cross-section area towards the pump outlet (at the top of the white pipe above)

In the dataset provided, other sensors not located on one of these component are positionned at the **pump** level.

**Let's load the content of each CSV file (we have one per component) and build a single CSV file with all the sensors:** we will obtain a dataset with 10 months of data (spanning from `2019-01-01` to `2019-10-27`) for 30 sensors (`Sensor0` to `Sensor29`) with a 1-minute sampling rate:

In [7]:
%%time

# Loops through each subfolder of the original dataset:
sensor_df_list = []
tags_description_dict = dict()
for root, dirs, files in os.walk(os.path.join(TMP_DATA, 'sensors-data')):
    # Reads each file and set the first column as an index:
    for f in files:
        print('Processing:', os.path.join(root, f))
        df = pd.read_csv(os.path.join(root, f))
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df = df.set_index('Timestamp')
        sensor_df_list.append(df)
        
        component = root.split('/')[-1]
        current_sensors = df.columns.tolist()
        current_sensors = dict(zip(current_sensors, [component] * len(current_sensors)))
        tags_description_dict = {**tags_description_dict, **current_sensors}
        
# Concatenate into a single dataframe:
equipment_df = pd.concat(sensor_df_list, axis='columns')
equipment_df = equipment_df.reset_index()
equipment_df['Timestamp'] = pd.to_datetime(equipment_df['Timestamp'])
equipment_df = equipment_df[[
    'Timestamp', 'Sensor0', 'Sensor1', 'Sensor2', 'Sensor3', 'Sensor4',
    'Sensor5', 'Sensor6', 'Sensor7', 'Sensor8', 'Sensor9', 'Sensor10',
    'Sensor11', 'Sensor24', 'Sensor25', 'Sensor26', 'Sensor27', 'Sensor28',
    'Sensor29', 'Sensor12', 'Sensor13', 'Sensor14', 'Sensor15', 'Sensor16',
    'Sensor17', 'Sensor18', 'Sensor19', 'Sensor20', 'Sensor21', 'Sensor22',
    'Sensor23'
]]

# Register a component for each sensor:
tags_description_df = pd.DataFrame.from_dict(tags_description_dict, orient='index')
tags_description_df = tags_description_df.reset_index()
tags_description_df.columns = ['Tag', 'Component']

print(equipment_df.shape)
equipment_df.head()

Processing: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/volute/component4_file1.csv
Processing: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/pump/component1_file1.csv
Processing: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/impeller/component2_file1.csv
Processing: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/shaft/component5_file1.csv
Processing: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/sensors-data/motor/component3_file1.csv
(86400, 31)
CPU times: user 801 ms, sys: 87.7 ms, total: 889 ms
Wall time: 952 ms


,Timestamp,Sensor0,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,...,Sensor14,Sensor15,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,Sensor22,Sensor23
0,2019-01-01 00:00:00,-0.828743,-0.068548,0.151633,-0.194587,0.862565,0.380223,6.852690,6.299292,7.219419,...,0.515517,0.565051,1.034995,1.296515,0.452947,0.548628,0.789443,0.863054,0.923071,0.973008
1,2019-01-01 00:05:00,0.019142,-0.946242,0.580651,-0.155384,0.116038,-0.185083,6.720679,7.688384,6.779484,...,0.637117,0.996548,0.493391,0.725938,0.896024,0.854002,0.600282,0.516597,0.192187,0.078018
2,2019-01-01 00:10:00,-0.036186,0.206872,0.343356,0.083626,-0.342867,-0.461456,7.581135,7.061488,6.230064,...,0.693980,0.287831,0.757481,1.254490,-0.040230,-0.229816,-0.488845,-0.712760,-0.752211,-0.933826
3,2019-01-01 00:15:00,-0.550611,-0.301188,0.148512,0.051228,-0.385147,-0.008690,7.486352,7.303732,6.881112,...,0.710251,1.224501,1.212985,1.196189,-0.918477,-0.968557,-0.881842,-0.860428,-0.714133,-0.515620
4,2019-01-01 00:20:00,-0.606425,-0.331560,-0.093932,0.784581,0.256889,-1.209056,7.010256,6.758775,7.089719,...,0.636829,0.845588,0.951010,2.279935,-0.460253,-0.338305,-0.064312,0.254336,0.443540,0.634771


In [8]:
equipment_df['Timestamp'] = pd.to_datetime(equipment_df['Timestamp'])
equipment_df = equipment_df.set_index('Timestamp')
equipment_df

,Sensor0,Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6,Sensor7,Sensor8,Sensor9,...,Sensor14,Sensor15,Sensor16,Sensor17,Sensor18,Sensor19,Sensor20,Sensor21,Sensor22,Sensor23
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,-0.828743,-0.068548,0.151633,-0.194587,0.862565,0.380223,6.852690,6.299292,7.219419,7.004522,...,0.515517,0.565051,1.034995,1.296515,0.452947,0.548628,0.789443,0.863054,0.923071,0.973008
2019-01-01 00:05:00,0.019142,-0.946242,0.580651,-0.155384,0.116038,-0.185083,6.720679,7.688384,6.779484,6.979925,...,0.637117,0.996548,0.493391,0.725938,0.896024,0.854002,0.600282,0.516597,0.192187,0.078018
2019-01-01 00:10:00,-0.036186,0.206872,0.343356,0.083626,-0.342867,-0.461456,7.581135,7.061488,6.230064,6.859606,...,0.693980,0.287831,0.757481,1.254490,-0.040230,-0.229816,-0.488845,-0.712760,-0.752211,-0.933826
2019-01-01 00:15:00,-0.550611,-0.301188,0.148512,0.051228,-0.385147,-0.008690,7.486352,7.303732,6.881112,6.877079,...,0.710251,1.224501,1.212985,1.196189,-0.918477,-0.968557,-0.881842,-0.860428,-0.714133,-0.515620
2019-01-01 00:20:00,-0.606425,-0.331560,-0.093932,0.784581,0.256889,-1.209056,7.010256,6.758775,7.089719,6.988612,...,0.636829,0.845588,0.951010,2.279935,-0.460253,-0.338305,-0.064312,0.254336,0.443540,0.634771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-27 23:35:00,0.572524,0.831853,0.012117,-0.170698,-0.468361,-0.938305,7.014236,6.712674,7.047559,7.205919,...,0.587945,1.357115,0.408277,0.454207,0.297795,0.437233,0.713052,0.802132,0.857221,0.850883
2019-10-27 23:40:00,0.100300,0.808269,-0.956786,0.374711,-1.346233,-0.949926,7.621383,6.781719,6.918772,7.916911,...,0.772307,1.976682,0.557193,1.073810,0.952974,0.911620,0.774939,0.702791,0.425559,0.176686
2019-10-27 23:45:00,-0.315199,0.077039,0.043951,-0.605731,1.199912,0.247149,7.405565,7.736269,6.358986,6.946860,...,1.280365,1.085853,1.077586,1.017124,0.256050,-0.012063,-0.231324,-0.513373,-0.566445,-0.779766


In [9]:
%%time

os.makedirs(os.path.join(TRAIN_DATA, 'centrifugal-pump'), exist_ok=True)
equipment_fname = os.path.join(TRAIN_DATA, 'centrifugal-pump', 'sensors.csv')
equipment_df.to_csv(equipment_fname)

CPU times: user 4.27 s, sys: 69.6 ms, total: 4.34 s
Wall time: 13.9 s


Let's also persist the tags description file as it will be useful when analyzing the model results:

In [10]:
tags_description_fname = os.path.join(TMP_DATA, 'tags_description.csv')
tags_description_df.to_csv(tags_description_fname, index=None)

## Loading label data
---
This dataset contains synthetically generated anomalies over different periods of time. Labels are stored as time ranges with a start and end timestamp. Each label is a period of time where we know some anomalous behavior happen:

In [11]:
label_fname = os.path.join(TMP_DATA, 'label-data', 'labels.csv')
labels_df = pd.read_csv(label_fname, header=None)
labels_df.to_csv(os.path.join(PROCESSED_DATA, 'label-data', 'labels.csv'), index=None, header=None)
labels_df.columns = ['start', 'end']
labels_df.head()

,start,end
0,2019-01-10 00:00:00,2019-01-11 00:00:00
1,2019-01-20 00:00:00,2019-01-21 00:00:00
2,2019-01-30 00:00:00,2019-01-31 00:00:00
3,2019-02-09 00:00:00,2019-02-10 00:00:00
4,2019-02-19 00:00:00,2019-02-20 00:00:00


## Uploading data to Amazon S3
---
Let's now load our training data and labels to Amazon S3, so that Lookout for Equipment can access them to train and evaluate a model.

In [12]:
train_s3_path = f's3://{BUCKET}/{PREFIX_TRAINING}centrifugal-pump/sensors.csv'
!aws s3 cp $equipment_fname $train_s3_path

label_s3_path = f's3://{BUCKET}/{PREFIX_LABEL}labels.csv'
!aws s3 cp $label_fname $label_s3_path

upload: ../data/processed/27b53ba8-baa0-4298-809a-0afe9384c280/training-data/centrifugal-pump/sensors.csv to s3://l4esitewisejjhj-asset1-train-inference/27b53ba8-baa0-4298-809a-0afe9384c280/training-data/centrifugal-pump/sensors.csv
upload: ../data/interim/27b53ba8-baa0-4298-809a-0afe9384c280/label-data/labels.csv to s3://l4esitewisejjhj-asset1-train-inference/27b53ba8-baa0-4298-809a-0afe9384c280/label-data/labels.csv


## Conclusion
---

### Enabling notifications in Jupyter Notebook

**Notify** is a Jupyter Notebook extension that notifies the user once a long-running cell has finished executing. It does so through browser notification. When you’ll run the below cell for the first time, your browser will ask you to allow notifications in your notebook. You should press ‘Yes.’

In [13]:
!pip install jupyternotify --quiet

%load_ext jupyternotify

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


<IPython.core.display.Javascript object>

In [14]:
# Needed for visualizing markdowns programatically
from IPython.display import display, Markdown

display(Markdown(
'''
<span style="color:green"><span style="font-size:50px">**Success!**</span></span>
<br/>
In this notebook, you downloaded the getting started dataset and prepared it for ingestion in Amazon Lookout for Equipment.

You also had a quick overview of the dataset with basic timeseries visualization.

You uploaded the training time series data and the anomaly labels to Amazon S3: in the next notebook of this getting started, you will be acquainted with the Amazon Lookout for Equipment API to create your first dataset. 

Move on to the next notebook.
'''))


<span style="color:green"><span style="font-size:50px">**Success!**</span></span>
<br/>
In this notebook, you downloaded the getting started dataset and prepared it for ingestion in Amazon Lookout for Equipment.

You also had a quick overview of the dataset with basic timeseries visualization.

You uploaded the training time series data and the anomaly labels to Amazon S3: in the next notebook of this getting started, you will be acquainted with the Amazon Lookout for Equipment API to create your first dataset. 

Move on to the next notebook.
